## Facebook OAuth 2.0 Flow with IPython Notebook

Facebook implements OAuth 2.0 as its standard authentication mechanism, and this example demonstrates how get an access token for making API requests once you've created an app and gotten a "client id" value that can be used to initiate an OAuth flow. Note that you will need an ordinary Facebook account in order to login, create an app, and get these credentials. 

You can create an app through the "Developer" section of your account settings as shown below or by navigating directly to https://developers.facebook.com/apps/. You will need the APP_ID from your app for the code below to work, and you will need to supply your own "_Valid OAuth redirect URIs_" (set from the "Advanced" Tab) to your app settings, to be the one that corresponds to your machine.


<img src="https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/fbapp.PNG">

<img src="https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/fbapp_advanced.PNG">

**You must ensure that your browser is not blocking popups in order for this script to work.**


In [ ]:
import requests
from flask import Flask, request
from threading import Timer
from IPython.display import display
from IPython.display import Javascript as JS

# XXX: Get this value from your Facebook application's settings for the OAuth flow
# at https://developers.facebook.com/apps

APP_ID = '1405218313107224' 

# This value is where Facebook will redirect. We'll configure an embedded
# web server to be serving requests here

REDIRECT_URI = 'http://ipython.ipeirotis.com:5000/oauth_helper'

# You could customize which extended permissions are being requested for your app
# by adding additional items to the list below. See
# https://developers.facebook.com/docs/reference/login/extended-permissions/

EXTENDED_PERMS = ['public_profile', 'user_friends']

# A temporary file to store a code from the web server

OAUTH_FILE = '/home/ubuntu/data/fb_access_token.txt'

# Configure an emedded web server that accepts one request, parses
# the fragment identifier out of the browser window redirects to another
# handler with the parsed out value in the query string where it can be captured
# and stored to disk. (A webserver cannot capture information in the fragment 
# identifier or that work would simply be done in here.)

webserver = Flask("FacebookOAuth")
@webserver.route("/oauth_helper")
def oauth_helper():
    return '''<script type="text/javascript">
    var at = window.location.hash.substring("access_token=".length+1).split("&")[0]; 
    setTimeout(function() { window.location = "/access_token_capture?access_token=" + at }, 1000 /*ms*/);
    </script>'''

# Parses out a query string parameter and stores it to disk. This is required because
# the address space that Flask uses is not shared with IPython Notebook, so there is really
# no other way to share the information than to store it to a file and access it afterward
@webserver.route("/access_token_capture")
def access_token_capture():
    access_token = request.args.get('access_token')
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(access_token)
    f.close()
    
    # It is safe (and convenient) to shut down the web server after this request
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return access_token


# Send an OAuth request to Facebook, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
    
args = dict(client_id=APP_ID, redirect_uri=REDIRECT_URI,
            scope=','.join(EXTENDED_PERMS), type='user_agent', display='popup'
            )

oauth_url = 'https://facebook.com/dialog/oauth?' + urllib.urlencode(args)

Timer(1, lambda: display(JS("window.open('%s')" % oauth_url))).start()


webserver.run(host='0.0.0.0')

access_token = open(OAUTH_FILE).read()

print access_token

After this step, you can use the access token as we described in our notes for the Jawbone API (and by learning, of course, the details of the [Facebook API](https://developers.facebook.com/docs/graph-api/reference/))


In [ ]:
import json

# Testing one of the API endpoints
endpoint = "https://graph.facebook.com/v2.2/me"

params = {}
headers = {'Authorization': 'Bearer ' + access_token}
resp = requests.get(endpoint, params=params, headers=headers)
userdata = json.loads(resp.text)
userdata

In [ ]:
# Testing one of the API endpoints
endpoint = "https://graph.facebook.com/v2.2/me/friends"

params = {}
headers = {'Authorization': 'Bearer ' + access_token}
resp = requests.get(endpoint, params=params, headers=headers)
data = json.loads(resp.text)
data